<a href="https://colab.research.google.com/github/zhangxs131/paper_demo_for_fun/blob/main/chinese_dialog_with_simctg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SimCTG Chinese Dialog Examples

使用SimCTG中已经训练好的LCCC数据集

In [1]:
#安装依赖

!git clone -q https://github.com/gmftbyGMFTBY/SimCTG-with-colab-demo.git
%cd /content/SimCTG-with-colab-demo
!pip install -r requirements.txt
%cd dialogue_generation

/content/SimCTG-with-colab-demo
     |████████████████████████████████| 60 kB 6.3 MB/s 
     |████████████████████████████████| 748.8 MB 16 kB/s 
     |████████████████████████████████| 5.9 MB 42.2 MB/s 
     |████████████████████████████████| 2.5 MB 69.6 MB/s 
     |████████████████████████████████| 1.2 MB 57.6 MB/s 
     |████████████████████████████████| 3.3 MB 27.8 MB/s 
     |████████████████████████████████| 880 kB 52.5 MB/s 
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=a426115ef4434a32560ce55f24d1f2733092ff45b120fba8c92c7d60c6eae390
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d92c309dbffcde21790a8737d4eefcbb1b430a2f95336ee11244efd31062e0d9
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built progressbar s

##加载模型

In [2]:
import torch
from simctgdialogue import SimCTGDialogue
# load model
model_path = r'cambridgeltl/simctg_lccc_dialogue'
eos_token, pad_token = '[SEP]', '[PAD]'
model = SimCTGDialogue(model_path, eos_token, pad_token)
tokenizer = model.tokenizer
model.eval()

Downloading:   0%|          | 0.00/923 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/521 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

PAD token exists.
[SEP]


SimCTGDialogue(
  (model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(21128, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inp

In [3]:
# 准备对话list
context_list = ['你觉得人活着是为了啥？', '为了享受生活，但享受之前得有钱呀', '是啊，没钱的话到处都不好搞', '那你说怎么能赚钱呢']

##使用Contrastive Search

In [4]:
#Contrastive Search

beam_width, alpha, decoding_len = 5, 0.6, 64
print (model.contrastive_search(context_list, beam_width, alpha, decoding_len))

不知道啊，就一个月工资而已


##使用其他方式解码

###diverse contrastive search

In [5]:
# use diverse contrastive search to generate the result，就是将对比搜索核核采样结果合并得到不同结果
sample_step, nucleus_p = 1, 0.95
beam_width, alpha, decoding_len = 5, 0.6, 64
print(model.diverse_contrastive_search(context_list, sample_step, nucleus_p, beam_width, alpha, decoding_len))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


赚钱就是一个目的，不要想太多


###核采样

In [6]:
nucleus_p, decoding_len = 0.95, 64
print(model.nucleus_sampling(context_list, nucleus_p, decoding_len))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


你都找好久工作了呀，你在武汉的话还是找个武汉某公司的


###Greedy Search

In [7]:
decoding_len = 64
print (model.greedy_search(context_list, decoding_len))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


你说的是什么？


###Beam Search

In [8]:
beam_width, decoding_len = 10, 64
print (model.beam_search(context_list, beam_width, decoding_len))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


赚钱就是为了享受生活，享受生活就是享受生活，享受生活就是享受生活，享受生活就是享受生活，享受生活就是享受生活，享受生活就是享受生


###总结

感觉综上结果来看，diverse contrastive search 解码效果好些，本次就是调用已经训练好的函数，如果需要自己在特定数据集上训练得到对话，生成故事，文本生成任务可以从 https://github.com/gmftbyGMFTBY/SimCTG-with-colab-demo 上进行实验，感觉给的demo示例很齐全。